In [1]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
#psi4.core.set_output_file('output.dat', False)
import time
from scipy.optimize import curve_fit


In [2]:
# setup basic arguments for qed-ci calculation

# z-matrix for CO
mol_str = """
C
O 1 1.1283
symmetry c1
"""

# options for Psi4
options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

# options for the PFHamiltonian Generator class - include cavity effects
cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 1, #<== this is a minimal photon basis, should explore increasing this 
}

# options for PFHamiltonian Generator class - exclude cavity effects
cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}



In [ ]:
# template for the z-matrix so we can scan through bond lengths
mol_tmpl = """
C
O 1 **R**
symmetry c1
"""


# number of bondlengths in the scan
N_R = 50

# number of electronic states to save
N_el = 10

r_array = np.linspace(1.0, 4.0, N_R)


# array for energies for LiH outside cavity
cavity_free_E_array = np.zeros((N_R, N_el))

# array  for energies inside the cavity
cavity_E_array = np.zeros((N_R, N_el))

r_idx = 0
for r in r_array:
    mol_str = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(mol_str)
    print(mol_str)
    
    # call psi4 RHF
    scf_e, wfn = psi4.energy('SCF/sto-3g', return_wfn=True)
    
    # call psi4 FCI
    fci_energy = psi4.energy('fci/sto-3g',ref_wfn=wfn)
    
    # LiH no cavity
    LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_free_dict)

    # check to make sure PFHamiltonian cavity-free FCI ground-state agrees with psi4
    assert np.isclose(LiH.CIeigs[0], fci_energy)
    
    # LiH with cavity
    LiH_cav = PFHamiltonianGenerator(mol_str, options_dict, cavity_dict)
    cavity_free_E_array[r_idx,:] = LiH.CIeigs[:N_el]
    cavity_E_array[r_idx,:] = LiH_cav.CIeigs[:N_el]
    r_idx += 1





C
O 1 1.0
symmetry c1


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  5, 4, 3
    Auxiliary basis highest AM E, G, H:  6, 5, 4
    Onebody   basis highest AM E, G, H:  6, 5, 4
    Solid Harmonics ordering:            gaussian

*** tstart() called on CHEMYM0V4HTALT
*** at Tue Nov  7 13:05:42 2023

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry C          line    61 file /Users/ptolley1/miniforge3/envs/work/share/psi4/basis/sto-3g.gbs 
    atoms 2 entry O          line    81 file /Users/ptolley1/miniforge3/envs/work/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         -------------------------------------

 Completed constant offset matrix in 1.350771188735962 seconds
 Completed Hamiltonian build in 1551.3925576210022 seconds

Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  5, 4, 3
    Auxiliary basis highest AM E, G, H:  6, 5, 4
    Onebody   basis highest AM E, G, H:  6, 5, 4
    Solid Harmonics ordering:            gaussian

*** tstart() called on CHEMYM0V4HTALT
*** at Tue Nov  7 13:39:54 2023

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry C          line    61 file /Users/ptolley1/miniforge3/envs/work/share/psi4/basis/sto-3g.gbs 
    atoms 2 entry O          line    81 file /Users/ptolley1/miniforge3/envs/work/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference

 Completed constant offset matrix in 1.3387751579284668 seconds
 Completed Hamiltonian build in 1588.5530281066895 seconds


In [ ]:
# z-matrix for CO
mol_str = """
C
O 1 1.1283
symmetry c1
"""

# options dictionary for the psi4 calculation
options_dict0 = {'basis': 'sto-3g',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-8,
               'd_convergence' : 1e-7
               }

# let's run a psi4 geometry optimization to get the optimal geometry

# set the options
psi4.set_options(options_dict0)

# first set the coordinates in init_string as the molecule's geometry 
mol0 = psi4.geometry(mol_str)

# now capture the psi4 geometry data with the guess coordinates
init_geometry_string = psi4.core.Molecule.create_psi4_string_from_molecule(mol0)

# run the geometry optimization
psi4.optimize("HF")

guess_geometry = mol0.geometry()

guess_string = psi4.core.Molecule.create_psi4_string_from_molecule(mol0)


In [ ]:
# options dictionary for the psi4 calculation
options_dict = {'basis': 'sto-3g',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-8,
               'd_convergence' : 1e-7
               }

# set the options
psi4.set_options(options_dict)

# first set the coordinates in guess_string as the molecule's geometry 
mol = psi4.geometry(guess_string)

# now capture the psi4 geometry data with the guess coordinates
guess_geometry_string = psi4.core.Molecule.create_psi4_string_from_molecule(mol)

# run the geometry optimization
psi4.optimize("HF")

opt_geometry = mol.geometry()

# capture the psi4 geometry data from the optimization
opt_geometry_string = psi4.core.Molecule.create_psi4_string_from_molecule(mol)

# run excited-state calculations - get first 5 excited states
n_states = 1

# run psi4 TDSCF at same level of theory as geometry optimization
psi4_rhf_e, wfn = psi4.frequency("CAM-b3lyp/sto-3g", return_wfn=True, molecule=mol,maxiter=50)



In [ ]:
from matplotlib import pyplot as plt
from scipy import interpolate

## Calculation of the Huang-Rhys Factor
Huang Rhys factor can be calculated by both

$$ S = 1/2(\Delta x / x_0)^2 \tag{Turner}$$

from the mode anharmonicity paper

and 

$$ S = \frac{m\omega_{vib} \Delta x^2}{2 \hbar} \tag{Hsu}$$

from the polaritonic Huang-Rhys factor paper

In [ ]:
# Constants and Variables
x_0 = wfn.frequency_analysis["Xtp0"].data[5]
k = wfn.frequency_analysis["k"].data[5] * 10 ** 2
min_S0_loc = np.argmin(cavity_free_E_array[:,0])
min_S1_loc = np.argmin(cavity_free_E_array[:,1])
delta_au = (r_array[min_S1_loc] - r_array[min_S0_loc]) * 1.8897
delta_m = (r_array[min_S1_loc] - r_array[min_S0_loc]) * 10 ** (-10)
mu_kg = (12.011 * 15.999 / (12.011 + 15.999)) * (10 ** (-3)) / (6.022 * 10 ** 23)
omega_vib = np.sqrt(k / mu_kg)
h_bar = (6.626 * 10 ** (-34)) / (2 * np.pi)

# Turner
S_Turner = 0.5 * (delta_au / x_0) ** 2

# Hsu
S_Hsu = mu_kg * omega_vib * delta_m ** 2 / (2 * h_bar)

print(S_Turner)
print(S_Hsu)

In [ ]:
alpha_inv_Angs = 2.5944
D_e = cavity_free_E_array[49,0] - cavity_free_E_array[min_S0_loc,0] 

V_r = D_e * (1 - np.exp(-alpha * (r_array - r_eq))) ** 2

V_r_spline = interpolate.UnivariateSpline(r_array, V_r, k=5)

# now define the expansion coefficients to expand the Morse potential as a polynomial
# harmonic
k = 2 * D_e * alpha ** 2
f_spline = V_r_spline.derivative()
k_spline = f_spline.derivative()
g_spline = k_spline.derivative()
h_spline = g_spline.derivative()

k_num = k_spline(r_eq)
g_num = g_spline(r_eq)
h_num = h_spline(r_eq)

print(k)
print(k_num)

V_H = 1/2 * k_num * (r_array - r_eq) ** 2

# cubic
g = -6 * D_e * alpha ** 3

V_C = V_H + 1/6 * g_num * (r_array - r_eq) ** 3

# quartic 
h = 14 * D_e * alpha ** 4

V_Q = V_H + V_C + 1/24 * h_num * (r_array - r_eq) ** 4


#plt.plot(r_array, V_r, 'red')
#plt.plot(r_array, V_r_spline(r), 'blue')
#plt.plot(r, V_C, 'green')
plt.plot(r_array, V_Q, 'purple')
#plt.xlim(0.75 * r_eq, 1.5 * r_eq)
#plt.ylim(0, 2e-18)
plt.show()
print(V_Q)

In [ ]:
def quartic_morse(r, De, k, re, g, h):
    return (1 / 2) * k * (r - re) ** 2 + (1 / 6) * g * (r - re) ** 3 + (1 / 24) * h * (r - re) ** 4
    #return De * (1 - np.exp(-k * (r - re))) ** 2 + g * (r - re) ** 3 + h * (r - re) ** 4

# Initial parameter guesses
initial_guess = [0.3, kappa, r_eq, g, h]

# Perform the curve fit
fit_params, covariance = curve_fit(quartic_morse, r_array, cavity_free_E_array[:,0], p0=initial_guess, maxfev=10000)

# Extract the optimized parameters
De_fit, k_fit, re_fit, g_fit, h_fit = fit_params

V_fit = quartic_morse(r_array, De_fit, k_fit, re_fit, g_fit, h_fit)

print(V_fit)

First plot the ground-state potential energy surfaces for LiH inside and outisde the cavity.  The effect of the cavity will raise the energy slightly.

In [ ]:
plt.plot(r_array, cavity_free_E_array[:,0], label='cavity free |g>')
#plt.plot(r_array, cavity_free_E_array[:,1], 'r-', label='cavity free |e>')
plt.plot(r_array, V_r_spline, label='fit')
plt.legend()
plt.show()


Next plot the ground state surface shifted by the cavity energy and the first excited state surface of 
LiH outside the cavity along with the lower- and upper-polariton surfaces from LiH inside the cavity.
Note that lower polariton surface intersects with another surface for the molecule in the cavity (see the crossing of the red and green curves around r = 1.5 Angstroms); the polariton surface is smooth, so we should really re-sort the eigenvalues around this point. Ruby has some experience doing this. 

In [ ]:
plt.plot(r_array, cavity_free_E_array[:,0]+0.12086, label='cavity free |g>+$\omega$')
plt.plot(r_array, cavity_free_E_array[:,1], label='cavity free |e>')
plt.plot(r_array, cavity_E_array[:,1], label='cavity |LP>')
plt.plot(r_array, cavity_E_array[:,2], label='cavity |LP>')
plt.plot(r_array, cavity_E_array[:,3], label='cavity |UP>')
plt.legend()
plt.show()